In [1]:
import pandas as pd
import os
import requests
import zipfile
import io
import random
import csv

def download_and_prepare_wtq(sample_size=50):
    print(">>> [Plan B] 正在直接从官方源下载 WikiTableQuestions 数据集...")
    
    # 1. 定义官方下载地址 (斯坦福 NLP 组的 GitHub)
    url = "https://github.com/ppasupat/WikiTableQuestions/archive/refs/heads/master.zip"
    output_dir = "benchmark_real"
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    try:
        # 2. 下载 Zip 包
        print("    正在下载 ZIP 包 (约 60MB)... 请耐心等待")
        r = requests.get(url)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        print("    下载完成，正在解压...")
        
        # 3. 解析数据
        # WTQ 的数据结构里，表格都在 csv/ 文件夹下
        # 我们还需要读取 data/pristine-unseen-tables.tsv 来获取对应的问题
        
        # 先解压所有 CSV 到临时目录
        z.extractall(output_dir)
        base_path = os.path.join(output_dir, "WikiTableQuestions-master")
        
        # 读取测试集的问题列表
        # 格式: id \t question \t table_id \t answer
        test_file_path = os.path.join(base_path, "data", "pristine-unseen-tables.tsv")
        
        print(f"    正在解析测试集问题列表: {test_file_path}")
        
        tasks = []
        
        # 读取 TSV 文件
        with open(test_file_path, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter='\t', fieldnames=['id', 'question', 'table_id', 'targetValue', 'possibleAnswers'])
            all_rows = list(reader)
            
        print(f"    原始测试集共有 {len(all_rows)} 条数据")
        
        # 4. 随机采样
        random.seed(42) # 保证论文可复现
        sampled_rows = random.sample(all_rows, sample_size)
        
        print(f"    正在准备 {sample_size} 个实验任务...")
        
        final_tasks = []
        
        for i, row in enumerate(sampled_rows):
            # 获取表格文件的路径
            # row['table_id'] 类似 "csv/200-csv/45.csv"
            # 我们需要把它拼接到解压路径上
            original_csv_path = os.path.join(base_path, row['table_id'])
            
            # 读取原始表格
            if os.path.exists(original_csv_path):
                # 复制并重命名表格到我们的 benchmark 目录，方便管理
                df = pd.read_csv(original_csv_path)
                
                new_csv_name = f"wtq_test_{i}.csv"
                new_csv_path = os.path.join(output_dir, new_csv_name)
                df.to_csv(new_csv_path, index=False)
                
                final_tasks.append({
                    "id": row['id'],
                    "file": new_csv_path, # 指向我们另存为的 CSV
                    "question": row['question'],
                    "original_answer": row['targetValue']
                })
            else:
                print(f"    ⚠️ 警告: 找不到表格文件 {original_csv_path}")

        # 5. 保存任务列表
        task_df = pd.DataFrame(final_tasks)
        task_df.to_csv("real_benchmark_tasks.csv", index=False)
        
        print("\n" + "="*50)
        print(f"✅ 成功！已准备 {len(final_tasks)} 条真实 WikiTableQuestions 数据。")
        print(f"📂 表格文件保存在: {output_dir}/")
        print(f"📝 任务列表保存在: real_benchmark_tasks.csv")
        print("="*50)
        print("👉 下一步: 请直接运行 run_benchmark_v2.py (确保它读取的是 real_benchmark_tasks.csv)")

    except Exception as e:
        print(f"❌ 发生错误: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    download_and_prepare_wtq(sample_size=30) # 先跑 30 个试试，稳定后再改大

>>> [Plan B] 正在直接从官方源下载 WikiTableQuestions 数据集...
    正在下载 ZIP 包 (约 60MB)... 请耐心等待
    下载完成，正在解压...
    正在解析测试集问题列表: benchmark_real\WikiTableQuestions-master\data\pristine-unseen-tables.tsv
    原始测试集共有 4345 条数据
    正在准备 30 个实验任务...

✅ 成功！已准备 30 条真实 WikiTableQuestions 数据。
📂 表格文件保存在: benchmark_real/
📝 任务列表保存在: real_benchmark_tasks.csv
👉 下一步: 请直接运行 run_benchmark_v2.py (确保它读取的是 real_benchmark_tasks.csv)
